### Creando un indicador de gasto per capita por provincia

##### Librerias

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

###### Poblacion

In [2]:
#Se lee la base de poblacion por distrito
population = pd.read_pickle("../data/raw/poblacion-provincia-periodo.pickle")

In [3]:
population

2007      2017 Ubigeo           2008   
Departamento Provincia                                                  
AMAZONAS     BAGUA              72757   74100.0   0102   72890.197317  \
             BONGARA            27465   25637.0   0103   27276.482449   
             CHACHAPOYAS        49700   55506.0   0101   50252.161055   
             CONDORCANQUI       43311   42470.0   0104   43226.155971   
             LUYA               48328   44436.0   0105   47923.931227   
...                               ...       ...    ...            ...   
TUMBES       ZARUMILLA          41054   48844.0   2403   41773.518103   
UCAYALI      ATALAYA            43933   49324.0   2502   44444.456868   
             CORONEL PORTILLO  333890  384168.0   2501  338606.421158   
             PADRE ABAD         52590   60107.0   2503   53297.318459   
             PURUS               3746    2860.0   2504    3646.259713   

                                        2009           2010           2011   
Departamento Provincia                                                       
AMAZONAS     BAGUA              73023.638481   73157.323938   73291.254134  \
             BONGARA            27089.258867   26903.320373   26718.658145   
             CHACHAPOYAS        50810.456554   51374.954649   51945.724251   
             CONDORCANQUI       43141.478147   43056.966202   42972.619812   
             LUYA               47523.240858   47125.900649   46731.882588   
...                                      ...            ...            ...   
TUMBES       ZARUMILLA          42505.646581   43250.606444   44008.622576   
UCAYALI      ATALAYA            44961.867988   45485.302677   46014.831060   
             CORONEL PORTILLO  343389.464942  348240.072441  353159.198038   
             PADRE ABAD         54014.150122   54740.622939   55476.866580   
             PURUS               3549.175093    3454.675429    3362.691895   

                                        2012           2013           2014   
Departamento Provincia                                                       
AMAZONAS     BAGUA              73425.429519   73559.850540   73694.517648  \
             BONGARA            26535.263424   26353.127510   26172.241762   
             CHACHAPOYAS        52522.835034   53106.357449   53696.362728   
             CONDORCANQUI       42888.438652   42804.422399   42720.570729   
             LUYA               46341.158898   45953.702036   45569.484687   
...                                      ...            ...            ...   
TUMBES       ZARUMILLA          44779.923805   45564.742966   46363.316977   
UCAYALI      ATALAYA            46550.524079   47092.453500   47640.691926   
             CORONEL PORTILLO  358147.809598  363206.888657  368337.430615   
             PADRE ABAD         56223.012459   56979.193759   57745.545452   
             PURUS               3273.157497    3186.007025    3101.177005   

                                        2015           2016           2018   
Departamento Provincia                                                       
AMAZONAS     BAGUA              73829.431293   73964.591926   74235.655967  \
             BONGARA            25992.597600   25814.186500   25461.029694   
             CHACHAPOYAS        54292.922896   54896.110775   56122.665021   
             CONDORCANQUI       42636.883320   42553.359851   42386.803447   
             LUYA               45188.479767   44810.660416   44064.472107   
...                                      ...            ...            ...   
TUMBES       ZARUMILLA          47175.886907   48002.698050   49700.046725   
UCAYALI      ATALAYA            48195.312806   48756.390441   49898.217526   
             CORONEL PORTILLO  373540.444934  378816.955335  389594.631775   
             PADRE ABAD         58522.204327   59309.309012   60915.419673   
             PURUS               3018.605654    2938.232831    2783.850182   

                                      

In [4]:
#
population['ubigeo'] = (population['Ubigeo'] # La variable ubigeo
                        .replace({'0701':'1501'}) # Se asigna la provincia del Callao a Lima Matropolitana
                        .apply(str) # Se asigna como string
                       )

In [5]:

populationpp = (population
                .drop(columns = ['Ubigeo']) #Se elimina la variable Ubigeo
                .groupby("ubigeo") # Se agrupa por la variable ubigeo
                .sum() #Se calcula el agregado
                .sort_index(axis = 1) #Se ordenan las columnas
               )

In [6]:
populationpr = (populationpp
                .assign(ubigeo = lambda df: df.index.str[0:2] + '00') # Se crea la variable de ubigeo por departamento
                .reset_index(drop = True) # Se resetea el index
                .groupby('ubigeo') # Se agrupa por el nuevo ubigeo
                .sum() #Se suma
               )

In [7]:
# Se concatenan ambas bases (equivalente a un append en stata)
populationb = pd.concat([populationpp,populationpr]).stack()

In [8]:
# Nombres para los indices
populationb.index.names = ['ubigeo','year']

###### Creando gasto per capita (a nivel de provincia y region)

In [9]:
#Se lee la base de poblacion por distrito
spending = pd.read_pickle("../data/raw/gastos-categorias-periodos.pickle")

In [10]:
# Se reemplaza Callao por Lima
spending['ubigeo'] = spending['ubigeo'].replace({'0701':'1501'})

In [11]:
# Solo gastos post 2006
spending = spending[spending.year > 2006]

In [12]:
data = (spending
        .groupby(['ubigeo','year','BtnTipoGobierno','TIPO_G']) #Agrupacion
        [['Devengado','PIM']] #Variables de devengado y PIM
        .sum() # Suma
        .rename(columns = lambda col: "USE_" + col) #Renombrando variables
        .join(populationb.rename("pop")) #Uniendo con la poblacion
        .apply(lambda y: y/y['pop'], axis = 1) #Diviendo las variables por la poblacion
        .drop(columns = ['pop']) #Eliminando poblacion
       )


##### Asignando gasto de region a nivel de provincia

In [13]:
# Separando regiones de provincias

dataprovince = data.loc[data.index.get_level_values(0).str[2:] != '00']
dataregion = data.loc[data.index.get_level_values(0).str[2:] == '00']


In [14]:
# Creando variable region

dataprovince.loc[:,'region'] = dataprovince.index.get_level_values(0).str[:2]
dataregion.loc[:,'region'] = dataregion.index.get_level_values(0).str[:2]

/tmp/ipykernel_7705/2166074236.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataprovince.loc[:,'region'] = dataprovince.index.get_level_values(0).str[:2]
/tmp/ipykernel_7705/2166074236.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataregion.loc[:,'region'] = dataregion.index.get_level_values(0).str[:2]


In [15]:
# Reordenando variables

dataprovince = (dataprovince
                .set_index('region',append= True) #Creando un nuevo indice
                .stack() #Columnas a indice (equivalente al reshape)
                .unstack([2,3,5]) # Indice a columnas
               )

dataregion = (dataregion
              .set_index('region',append= True)  #Creando un nuevo indice
              .stack() #Columnas a indice (equivalente al reshape)
              .unstack([2,3,5]) #Indice a columnas
              .reset_index('ubigeo', drop = True) #Quitando un indice
             )

In [16]:
data = dataprovince.join(dataregion)

data = data.sort_index(axis = 1).fillna(0)

In [17]:
data['period'] = pd.cut(data.index.get_level_values('year'), [2007, 2012, 2017, 2022], include_lowest=False)

In [18]:
data_agregada = data.groupby(['ubigeo','period']).mean()

In [20]:
data_agregada.to_pickle("../data/interim/data_spending.pickle")